In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *
from fastai.column_data import *

In [2]:
path='data/ml-latest-small/'

In [3]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## Fastai way

In [31]:
val_idxs = get_cv_idxs(len(ratings))
weight_decay = 2e-4
number_of_embedding_factors = 1000
batch_size=64

len(val_idxs)

20000

In [25]:
cf = CollabFilterDataset.from_csv(path, 'ratings.csv', 'userId', 'movieId', 'rating')

In [26]:
learn = cf.get_learner(number_of_embedding_factors, val_idxs, bs=batch_size, opt_fn=optim.Adam)

In [27]:
learn.fit(1e-2, 2, wds=weight_decay, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                 
    0      0.982943   0.910231  
    1      1.05116    1.023799                                
    2      0.625447   0.799332                                 



[0.7993321]

### Predict from the real dataset

You need to load the full dataset in. 
The fastai learner class already has the validation set loaded so you need you need to override this with the full set of data


In [44]:
# get a dataframe and then create a Dataloader so the model can do batched loading
df = CollabFilterDataset.from_csv(path, 'ratings.csv', 'userId', 'movieId', 'rating')
dl = DataLoader(df)

In [40]:
predictions = learn.predict_dl(dl)

In [41]:
predictions
len(predictions)

100004

In [42]:
ratings["predicted_rating"] = predictions

#### DF with predictions !!

In [47]:
ratings.head(15)

,userId,movieId,rating,timestamp,predicted_rating
0,1,31,2.5,1260759144,2.587995
1,1,1029,3.0,1260759179,2.608524
2,1,1061,3.0,1260759182,2.852380
3,1,1129,2.0,1260759185,2.289287
4,1,1172,4.0,1260759205,3.577430
5,1,1263,2.0,1260759151,2.526428
6,1,1287,2.0,1260759187,2.913350
7,1,1293,2.0,1260759148,2.710319
8,1,1339,3.5,1260759125,2.788491
9,1,1343,2.0,1260759131,2.676279
